Last Updated - 11/14/2016

Environmental Context Analysis for Mar 02 02:00 case

In [1]:
from ipy import *
from mpl_toolkits.basemap import Basemap
import mpl_toolkits.mplot3d as mplot3d
import PIL
from mpl_toolkits.mplot3d import Axes3D
from matplotlib._png import read_png
from scipy.interpolate import interp1d

# Specify file locations

In [2]:
#############################################
##   MEC Files
#############################################
#Mar 02
MEC_file = 'Z:/mms1/mec/srvy/l2/epht89d/2016/03/mms1_mec_srvy_l2_epht89d_20160302_v2.1.0.cdf'

In [3]:
#############################################
##   SC Potential Files
#############################################
#Mar 02
SC_Pot_file = 'Z:/mms1/edp/fast/l2/scpot/2016/03/mms1_edp_fast_l2_scpot_20160302000000_v2.1.3.cdf'

In [4]:
#############################################
##   OMNI IMF Data Files
#############################################
#Oct 26
Omni_context_file = 'C:/Users/cschiff/Documents/GitHub/PAD/Mar 02 - pictures/Mar_02_omni2_6314.lst'

# Open Files

In [5]:
CDFs           = {}
CDFs['MEC']    = pycdf.CDF(MEC_file)
CDFs['SC_Pot'] = pycdf.CDF(SC_Pot_file)

In [6]:
Data               = {}
Data['IMF']        = open(Omni_context_file,'r')

# Shared Constants and Variables

In [7]:
date_format = '%b %d %Y'
time_format = '%H:%M:%S'
Re          = 6378.14

# IMF Context

## read omni context data

In [8]:
omni_blob = Data['IMF'].readlines()
Data['IMF'].close()
num_rows = len(omni_blob)
num_cols = len(omni_blob[0].strip('\n').split())-3
data     = np.zeros((num_rows,num_cols))
dates    = []

counter  = 0
for omni in omni_blob:
    #first get the data as an array
    o                = omni.strip('\n').split()
    current_datetime = dt.datetime(int(o[0]),1,1,int(o[2])) + dt.timedelta(int(o[1])-1)
    dates.append(current_datetime)
    data[counter,:]  = map(float,o[3:])
    counter          = counter + 1    

## plot combined 'weather report'

In [13]:
fig_w = plt.figure(69,figsize=(7,9))
start_time_index = 2
stop_time_index  = 5
time_slice_IMF   = slice(start_time_index,stop_time_index+1)
time_step        = dt.timedelta(seconds = 30*60) 
#define the Epoch labels
x_labels = [dt.datetime.strftime(dates[start_time_index] + i*time_step,time_format) for i in range(0,2*(stop_time_index - start_time_index)+1)]

#IMF 
ax1_w = fig_w.add_subplot(3,1,1)
ax1_w.plot(dates[time_slice_IMF],data[time_slice_IMF,1],label = 'Bx')
ax1_w.plot(dates[time_slice_IMF],data[time_slice_IMF,2],label = 'By')
ax1_w.plot(dates[time_slice_IMF],data[time_slice_IMF,3],label = 'Bz')
ax1_w.plot(dates[time_slice_IMF],data[time_slice_IMF,0],label='B',color='k')
ax1_w.legend(loc='upper right')
ax1_w.minorticks_on()
ax1_w.set_ylabel('Magnetic Field (nT)',fontsize=12)
ax1_w.set_xticklabels([])

#solar wind
ax21_w = fig_w.add_subplot(3,1,2)
ax22_w = ax21_w.twinx()
ax21_w.plot(dates[time_slice_IMF],data[time_slice_IMF,5],'ko-',label = r'$n_p$')
ax22_w.plot(dates[time_slice_IMF],data[time_slice_IMF,7],'ro-',label = 'P')
ax21_w.legend(loc='lower left')
ax22_w.legend(loc='upper right')
ax21_w.minorticks_on()
ax22_w.minorticks_on()
ax21_w.set_ylabel('Proton number density',fontsize=12)
ax22_w.set_ylabel('Dynamic Pressure (nPa)',fontsize = 12)
ax21_w.set_xticklabels([])
ax22_w.set_xticklabels([])

#DST and Ap indices
ax31_w = fig_w.add_subplot(3,1,3)
ax32_w = ax31_w.twinx()
ax31_w.plot(dates[time_slice_IMF],data[time_slice_IMF,8],'ko-',label = 'DST')
ax32_w.plot(dates[time_slice_IMF],data[time_slice_IMF,9],'ro-',label = r'$a_p$')
ax31_w.legend(loc='lower left')
ax32_w.legend(loc='upper right')
ax31_w.minorticks_on()
ax31_w.set_ylabel('DST Value (nT)',fontsize=12)
ax32_w.set_ylabel(r'$a_p$ index value',fontsize=12)
ax32_w.set_xticklabels(x_labels)

fig_w.subplots_adjust(wspace=0.0,hspace=0.1)
plt.gcf().autofmt_xdate()
start_date = dt.datetime.strftime(dates[start_time_index],date_format)
start_time = dt.datetime.strftime(dates[start_time_index],time_format)
stop_time  = dt.datetime.strftime(dates[stop_time_index],time_format)
ax1_w.set_title('Environmental Conditions for %s %s-%s' % (start_date,start_time,stop_time))

plt.show()

# Footprints & Field Type & Spacecraft GSM Position

In [14]:
North_latlon = np.asarray(CDFs['MEC']['mms1_mec_pfn_geod_latlon'])
South_latlon = np.asarray(CDFs['MEC']['mms1_mec_pfs_geod_latlon'])

In [27]:
time_index_MEC_min   = 300
time_index_MEC_max   = 500
time_slice_MEC       = slice(time_index_MEC_min,time_index_MEC_max)
terminator_index_MEC = 330
delta_time_MEC       = time_index_MEC_max - time_index_MEC_min
time_step_MEC        = 40
Re                   = 6378.14

In [28]:
terminator_date = CDFs['MEC']['Epoch'][terminator_index_MEC]
# draw map with markers for float locations
m = Basemap(projection='mill',lon_0=0)
m.drawcoastlines()
m.drawparallels(np.arange(-90,90,15),labels=[1,0,0,0])
m.drawmeridians(np.arange(m.lonmin,m.lonmax+30,60),labels=[0,0,0,1])
nlons = North_latlon[time_slice_MEC,1]
nlats = North_latlon[time_slice_MEC,0]
slons = South_latlon[time_slice_MEC,1]
slats = South_latlon[time_slice_MEC,0]
Nx, Ny = m(nlons,nlats)
Sx, Sy = m(slons,slats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#ffffff',lake_color='#99ffff')
CS=m.nightshade(terminator_date)
m.scatter(Nx,Ny,3,marker='o',color='r')
m.scatter(Sx,Sy,3,marker='o',color='r')
plt.title('Day/Night Map for %s (UTC)' % terminator_date.strftime("%d %b %Y %H:%M:%S"))
plt.show()
print('Start date of scan: ',CDFs['MEC']['Epoch'][time_slice_MEC.start])
print('Stop date of scan:  ',CDFs['MEC']['Epoch'][time_slice_MEC.stop])
print(Nx.shape)

Start date of scan:  2016-03-02 02:30:00
Stop date of scan:   2016-03-02 04:10:00
(200L,)


In [29]:
plt.plot(CDFs['MEC']['mms1_mec_fieldline_type'][time_slice_MEC])
x        = np.arange(0,delta_time_MEC,time_step_MEC)
x_labels = []
for px in x:
    x_labels.append(dt.datetime.strftime(CDFs['MEC']['Epoch'][px+time_index_MEC_min],time_format))
plt.xticks(x,x_labels,rotation='vertical')    
y        = np.arange(-3,5,1)
y_labels = ['Bad Trace','Unreachable','Inside Earth','IMF','Closed','Open North','Open South','']
plt.yticks(y,y_labels)
plt.title('Field Line Type for %s (UTC)' % terminator_date.strftime("%d %b %Y"))
plt.show()

# Spacecraft position

In [30]:
sc_pos = np.array(CDFs['MEC']['mms1_mec_r_gsm'][time_index_MEC_min]/Re)
print(sc_pos)

[ 3.93438459 -7.32085316 -4.85580737]


# Spacecraft Potential

In [31]:
CDFs['SC_Pot'].keys()

['mms1_edp_epoch_fast_l2',
 'mms1_edp_label1_fast_l2',
 'mms1_edp_scpot_fast_l2',
 'mms1_edp_psp_fast_l2',
 'mms1_edp_dcv_fast_l2',
 'mms1_edp_bitmask_fast_l2',
 'mms1_edp_quality_fast_l2',
 'mms1_edp_deltap_fast_l2']

In [36]:
time_index_min = 250000
time_index_max = 350000
delta_time     = time_index_max - time_index_min
time_step      = 10000
time_slice     = slice(time_index_min,time_index_max)
plt.plot(CDFs['SC_Pot']['mms1_edp_scpot_fast_l2'][time_slice])
x        = np.arange(0,delta_time,time_step)
x_labels = []
for px in x:
    x_labels.append(dt.datetime.strftime(CDFs['SC_Pot']['mms1_edp_epoch_fast_l2'][px+time_index_min],time_format))
plt.xticks(x,x_labels,rotation='vertical')
plt.title('Spacecraft Potential for %s' % \
          dt.datetime.strftime(CDFs['SC_Pot']['mms1_edp_epoch_fast_l2'][time_index_min],date_format))
plt.show()

# Field Line Tracing - Something of an Art

In [38]:
Fieldline_file = 'C:/Users/cschiff/Documents/GitHub/PAD/Mar 02 - pictures/Mar_02_02_45_00_fieldline_contiguous.txt'
Fieldline      = open(Fieldline_file,'r')
fld_blob       = Fieldline.readlines()
Fieldline.close()
fld            = np.zeros((len(fld_blob),3))

counter  = 0
for f in fld_blob:
    fld[counter,:] = map(float,f.strip('\r\n').split())
    counter        = counter + 1

In [40]:
# Setup 3D Plot --------------------------------------------------------
fig = plt.figure(figsize=(10, 10)) 
ax = fig.add_subplot(111, projection='3d')

fld_limit = np.ceil(np.max(np.abs(fld)))

limits = [-fld_limit,fld_limit]
ax.set_xlim(limits)
ax.set_ylim(limits)
ax.set_zlim(limits)

ax.set_xlabel(r'$X_{GSM}$')
ax.set_ylabel(r'$Y_{GSM}$')
ax.set_zlabel(r'$Z_{GSM}$')

#plot sun direction
ax.quiver(1,0,0,1,0,0,color='#c4c67f',length=fld_limit/3,linewidth=2,pivot='tail')

# Plot Earth -----------------------------------------------------------
# The approach is to load the blue marble image into the an image of RGB
# tuples, with each of R, G, B being in [0, 1]. This is used as the
# coloring for a surface plot of a sphere.
bm = PIL.Image.open('C:/Users/cschiff/Documents/GitHub/PAD/Blue Marble/bluemarble.jpg')
bm = np.array(bm.resize([int(d/5) for d in bm.size]))
bm = bm.astype(float) / 257.

lons = np.linspace(-180, 180, bm.shape[1])       * np.pi / 180 
lats = np.linspace( -90,  90, bm.shape[0])[::-1] * np.pi / 180 

x = np.outer(np.cos(lons), np.cos(lats)).T
y = np.outer(np.sin(lons), np.cos(lats)).T
z = np.outer(np.ones(np.size(lons)), np.sin(lats)).T

ax.plot_surface(x, y, z, rstride=4, cstride=4, facecolors=bm)

# Plot Magnetic field lines. -------------------------------------------
# The magnetic field lines data needs to be converted to a rectangular
# coordinate system where the earth is centered at (0, 0, 0) and has a
# radius of 1.
#
qp = [fld_limit]
qm = [-fld_limit]
nf = len(fld)

ax.plot(fld[:,0],fld[:,1],fld[:,2],'r-')
ax.plot(qm*nf   ,fld[:,1],fld[:,2],'b-')
ax.plot(fld[:,0],qp*nf,   fld[:,2],'b-')
ax.plot(fld[:,0],fld[:,1],qm*nf   ,'b-')

#plot spacecraft location
ax.plot([sc_pos[0]]*2,[sc_pos[1]]*2,[sc_pos[2]]*2,'ko')
ax.plot(qm*2         ,[sc_pos[1]]*2,[sc_pos[2]]*2,'ko')
ax.plot([sc_pos[0]]*2,qp*2,         [sc_pos[2]]*2,'ko')
ax.plot([sc_pos[0]]*2,[sc_pos[1]]*2,qm*2,       'ko')

ax.text(sc_pos[0],sc_pos[1],sc_pos[2],'MMS1')
ax.text(3,0,0,'sunward')
plt.show()
